In [1]:
import pandas as pd
import mxnet as mx
from mxnet.gluon import nn
from mxnet import ndarray as nd

project_path = '/home/carsmart/users/xiaoming/Distracted_Driver_Detection/'
features_path = project_path + 'features/'
test_path_list = project_path + 'test_lst/test_test.lst'
model_save_path = project_path + 'model/'
predict_save_path = project_path + 'predict/'
ctx = mx.gpu(0)

## load test features

In [2]:
model_names = ['vgg11', 'densenet161', 'squeezenet1.0']
features_test = [nd.load(features_path + 'features_test_%s.nd' % model_name)[0] for model_name in model_names]
features_test = nd.concat(*features_test, dim = 1)

## predict

In [3]:
def get_net():
    net = nn.HybridSequential()
    with net.name_scope():
        net.add(
            nn.Dense(256, activation = 'relu'), 
            nn.Dropout(0.5), 
            nn.Dense(10)
        )
    net.hybridize()
    net.initialize(ctx = ctx)
    return net

net = get_net()
net.load_params(model_save_path + '9947.params', ctx = ctx)

In [5]:
output = nd.softmax(net(features_test.as_in_context(ctx))).asnumpy()
df_pred = pd.read_csv(project_path + 'sample_submission.csv')
test_pictures = pd.read_table(test_path_list, header = None)[2]
df_pred['img'] = test_pictures
for i, j in enumerate(df_pred.columns[1:]):
    df_pred[j] = output[:, i]

df_pred = df_pred.sort_values('img')
df_pred.to_csv(predict_save_path + 'pred.csv', index = None)